In [1]:
import json
import re

import pandas as pd

# 데이터프레임 전처리

In [2]:
def df_preprocess(df):
    print("기존 데이터 수 > ", len(df))
    df = delete_duplicte(df)
    df = delete_null(df)
    df = create_title(df)

    ## 한국어가 일정 비율 아래이면 kill
    df = df[df['text'].apply(sep)]
    df = df[df['title'].apply(sep)]
    print("한국어 비율 적은 데이터 제거 후 데이터 수 > ", len(df))

    ## death note 기반 kill
    df = df[df['author'].apply(kill_author)]
    df = df[df['text'].apply(kill_keyword)]
    print("DEATH NOTE 후 데이터 수 > ", len(df))

    ## 길이 기반 kill
    df = df[df['text'].apply(len) > 500]
    print("500자 이하 데이터 제거한 후 데이터 수 > ", len(df))

    ## 중국어 일본어 kill
    df = df[df['text'].apply(detect_chinese_japanese)]
    print("중국어 일본어 제거한 데이터 수 > ", len(df))
    return df


def delete_duplicte(df):
    df = df.drop_duplicates(["text"], keep='first')
    print("중복 제거 후 데이터 수 > ", len(df))
    return df


def delete_null(df):
    df = df.dropna(axis=0, subset=['text'])
    df = df[df['text'].apply(len) > 0]
    print("본문 없는 데이터 제거 후 데이터 수 > ", len(df))
    return df


def create_title(df):
    df.loc[df['title'].isnull(), 'title'] = df[df['title'].isnull()]['text'].apply(lambda x: x.split(".")[0])
    return df


def sep(text):
    try:
        return len(re.sub(r"[[^ㄱ-ㅎㅏ-ㅣ가-힣 \n 1-9]", "", text)) / len(text) * 100 < 20
    except:
        return False


def kill_author(author):
    death_author = ["@karis", "@dkgo98"]
    return False if author in death_author else True


def kill_keyword(text):
    death_keyword = ["코로나", "브런치"]
    for k in death_keyword:
        if k in text:
            return False
    return True


def detect_chinese_japanese(text: str) -> bool:
    return len(re.findall(r'[\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff]', text)) == 0


def delete_tag(text):
    text_split = text.split("\n")
    while text_split and len(text_split[-1]) < 6:
        text_split.pop()
    return "\n".join(text_split)

## 오거서

In [3]:
# fcb1 = pd.read_excel("./bookathon_common_dataset/2015 독서리뷰-수정.xlsx", header=1)
# fcb2 = pd.read_excel("./bookathon_common_dataset/2016 독서리뷰-수정.xlsx", header=1)
# fcb3 = pd.read_excel("./bookathon_common_dataset/2017 독서리뷰-수정.xlsx", header=1)
# fcb4 = pd.read_excel("./bookathon_common_dataset/2018 독서리뷰-수정.xlsx", header=1)
# fcb5 = pd.read_excel("./bookathon_common_dataset/2019-2020 독서리뷰-수정.xlsx", header=1)
# fcb6 = pd.read_excel("./bookathon_common_dataset/2021-2022.12.12 오거서 독서리뷰.xls", header=1)

# total_fcb = pd.concat([fcb1, fcb2, fcb3, fcb4, fcb5, fcb6])

# fcb_df = total_fcb[["도서명(또는 글제목)", "글 내용" ,"작성자명"]]
# fcb_df = fcb_df.rename(columns={"도서명(또는 글제목)": "title","글 내용": "text", "작성자명": "author"})

# fcb_df['text'] = fcb_df['text'].apply(str)
# fcb_df['title'] = fcb_df['title'].apply(str)
# fcb_df['author'] = fcb_df['author'].apply(str)
# fcb_df["url"] = np.NaN
# fcb_df["class"] = "감상&비평"

# fcb_df = df_preprocess(fcb_df)
# fcb_df.info()

## 브런치

In [4]:
# keyword = ['산문', '에세이', '인생', '일상에세이', '감성에세이', '감성에세이1', '수필']
keyword = ["수필", "에세이", "일상에세이"]

death_keyword = ["코로나", "브런치"]
death_author = []

In [5]:
total_brunch = []

for k in keyword:
    total_brunch += json.load(open(f"./brunch/{k}.json", "r", encoding="UTF8"))['data']

brunch_df = pd.DataFrame(data=total_brunch)
brunch_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28082 entries, 0 to 28081
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   28082 non-null  object
 1   title   28082 non-null  object
 2   author  28082 non-null  object
 3   text    28082 non-null  object
 4   url     28082 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


In [6]:
brunch_df = df_preprocess(brunch_df)
brunch_df.info()

기존 데이터 수 >  28082
중복 제거 후 데이터 수 >  27296
본문 없는 데이터 제거 후 데이터 수 >  27295


C:\Users\DIAL\AppData\Local\Temp\ipykernel_25748\2409298590.py:47: FutureWarning: Possible nested set at position 1
  return len(re.sub(r"[[^ㄱ-ㅎㅏ-ㅣ가-힣 \n 1-9]", "", text)) / len(text) * 100 < 20


한국어 비율 적은 데이터 제거 후 데이터 수 >  24778
DEATH NOTE 후 데이터 수 >  21624
500자 이하 데이터 제거한 후 데이터 수 >  19429
중국어 일본어 제거한 데이터 수 >  17791
<class 'pandas.core.frame.DataFrame'>
Int64Index: 17791 entries, 0 to 28081
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   17791 non-null  object
 1   title   17791 non-null  object
 2   author  17791 non-null  object
 3   text    17791 non-null  object
 4   url     17791 non-null  object
dtypes: object(5)
memory usage: 834.0+ KB


## 글틴

In [7]:
teen1 = json.load(open("./brunch/글틴수필.json", "r", encoding="UTF8"))
teen2 = json.load(open("./brunch/명예의전당.json", "r", encoding="UTF8"))

total_teen = teen1['data'] + teen2['data']
teen_df = pd.DataFrame(data=total_teen)
teen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4015 entries, 0 to 4014
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   4015 non-null   object
 1   title   4015 non-null   object
 2   author  4015 non-null   object
 3   text    4015 non-null   object
 4   url     4015 non-null   object
dtypes: object(5)
memory usage: 157.0+ KB


In [8]:
teen_df = df_preprocess(teen_df)
teen_df.info()

기존 데이터 수 >  4015
중복 제거 후 데이터 수 >  3902
본문 없는 데이터 제거 후 데이터 수 >  3901
한국어 비율 적은 데이터 제거 후 데이터 수 >  3598
DEATH NOTE 후 데이터 수 >  3564
500자 이하 데이터 제거한 후 데이터 수 >  3251
중국어 일본어 제거한 데이터 수 >  2938
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2938 entries, 0 to 4013
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   2938 non-null   object
 1   title   2938 non-null   object
 2   author  2938 non-null   object
 3   text    2938 non-null   object
 4   url     2938 non-null   object
dtypes: object(5)
memory usage: 137.7+ KB


# 데이터 전처리

In [9]:
total_df = pd.concat([brunch_df, teen_df])
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20729 entries, 0 to 4013
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   20729 non-null  object
 1   title   20729 non-null  object
 2   author  20729 non-null  object
 3   text    20729 non-null  object
 4   url     20729 non-null  object
dtypes: object(5)
memory usage: 971.7+ KB


In [11]:
def preprocess(x: str):
    # Remove emails
    x = re.sub(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", "", x).strip()

    # Remove URLs
    x = re.sub(r"(https?://)?(www\.)?[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(?:\.[a-zA-Z0-9](?:[a-zA-Z0-9-]"
               r"{0,61}[a-zA-Z0-9])?)+(?:/.*)?", "", x).strip()

    # Replace some invisible Unicode characters
    x = x.replace("\u2060", "").strip()

    # Remove hashtags
    x = re.sub(r"#\S+", "", x).strip()

    # Remove mentions
    x = re.sub(r"@\w+", "", x).strip()

    # Replace curly quotes
    x = x.replace("‘", "'").replace("’", "'").replace("“", '"').replace("”", '"')

    # Replace ellipsis
    x = x.replace("…", "...")

    # Replace em dash
    x = x.replace("—", "-")

    # Replace en dash
    x = x.replace("–", "-")

    # Replace 《 》
    x = x.replace("《", "<").replace("》", ">")

    # Replace 〈 〉
    x = x.replace("〈", "<").replace("〉", ">")

    # Replace 〔 〕
    x = x.replace("〔", "[").replace("〕", "]")

    # Replace 〖 〗
    x = x.replace("〖", "[").replace("〗", "]")

    # Replace 〘 〙
    x = x.replace("〘", "[").replace("〙", "]")

    # Replace 〚 〛
    x = x.replace("〚", "[").replace("〛", "]")

    # Replace 「 」
    x = x.replace("「", "\"").replace("」", "\"")

    # Replace 『 』
    x = x.replace("『", "\"").replace("』", "\"")

    # Replace 【 】
    x = x.replace("【", "[").replace("】", "]")

    # Remove nested parenthesis
    x = re.sub(r"[(\[{].*?[)\]}]|[)\]}]", "", x).strip()

    # Remove leftover parenthesis just in case
    x = re.sub(r"[()\[\]{}]", "", x).strip()

    # Remove Korean consonants and vowels
    x = re.sub(r"[ㄱ-ㅎㅏ-ㅣ]", "", x).strip()

    # Remove punctuations repeated twice or more
    x = re.sub(r"([@#$%^&*\-=+\\|/]){2,}", r"", x).strip()

    # Replace full stops repeated four times or more with three full stops
    x = re.sub(r"\.{4,}", "...", x).strip()

    # Replace question marks and exclamation marks repeated twice or more with one question mark
    x = re.sub(r"([?!]){2,}", r"\1", x).strip()

    # Replace \n with space
    x = re.sub(r"\n", " ", x).strip()

    # Replace multiple spaces with one space
    x = re.sub(r"\s+", " ", x).strip()

    return x

In [12]:
total_df['text'] = total_df['text'].apply(preprocess)
total_df['title'] = total_df['title'].apply(preprocess)
total_df.head(3)

,class,title,author,text,url
0,수필,370. 대학교수 김병조,@shrhim,한때 개그맨으로 잘 나가던 김병조 씨가 대학교수가 되어 회사를 찾았다. 회사 '명사...,https://brunch.co.kr/@shrhim/419
1,수필,연필로 쓰기,@webtutor,나는 여론을 일으키거나 거기에 붙어서 편을 끌어모으려는 목표를 가지고 있지 않다. ...,https://brunch.co.kr/@webtutor/604
2,수필,"아이고, 건망증",@naya1960,"자신이 어떤 사람인지를 모른다는 것은 인생의 수레를 안개 속에 굴러가게 해놓고, 말...",https://brunch.co.kr/@naya1960/196


In [13]:
total_df

,class,title,author,text,url
0,수필,370. 대학교수 김병조,@shrhim,한때 개그맨으로 잘 나가던 김병조 씨가 대학교수가 되어 회사를 찾았다. 회사 '명사...,https://brunch.co.kr/@shrhim/419
1,수필,연필로 쓰기,@webtutor,나는 여론을 일으키거나 거기에 붙어서 편을 끌어모으려는 목표를 가지고 있지 않다. ...,https://brunch.co.kr/@webtutor/604
2,수필,"아이고, 건망증",@naya1960,"자신이 어떤 사람인지를 모른다는 것은 인생의 수레를 안개 속에 굴러가게 해놓고, 말...",https://brunch.co.kr/@naya1960/196
4,수필,15. 아내에게 좋은 사람?,@eulachacha,"""좋은 사람이 되어야 좋은 사람을 만나게 된다."" 요즘 읽고 있는 <태도의 말들>에...",https://brunch.co.kr/@eulachacha/56
5,수필,양치기 소녀,@soganga4,# 1. 우리 가족은 내가 초등학교 2학년 여름방학이 되면서 강원도르 이사를 했다....,https://brunch.co.kr/@soganga4/62
...,...,...,...,...,...
4000,소설,우주 이야기,김영빈,1 너의 우주를 보고 싶어. 펭이 말했다. 목소리가 작아서 환청을 들은 것 같았다....,https://teen.munjang.or.kr/archives/86932
4002,소설,포도나무가 자라는 땅,김영빈,"벌써 넉 달째, 아무 말 없이 통장에 돈을 부치고 있는 사람이 누군지를 우는 생각했...",https://teen.munjang.or.kr/archives/86702
4008,시,한 아이가 슈퍼맨이 술을 마신 채 개처럼 벤치에 뻗은 걸 봤데,김영빈,한 아이가 슈퍼맨이 술을 마신 채 개처럼 벤치에 뻗은 걸 봤데 그 아이는 하루 종일...,https://teen.munjang.or.kr/archives/85812
4012,소설,빨간 줄무늬,김영빈,━ 줄무늬에는 힘이 있다. 선의 경계 안에 아무것도 침범할 수 없는 견고함. 밋밋하...,https://teen.munjang.or.kr/archives/84464


In [18]:
total_json = total_df.to_dict(orient="records")
json.dump(total_json, open("./data/hyunsoo_brunch_gteen.json", "w", encoding="utf-8"), ensure_ascii=False,
          indent=2)